###  Attempting to create loop

Ideas: 

    - Import the school district area shapefile, it has coordinates that be iterated and find all the census tracts for it
        - Only issue what to do about overlaps?? some school districts will be in the same county
        -- should map that

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import itertools

In [2]:
# Load county school district 

In [2]:
school_district = gpd.read_file('data/SchoolSites1920.shp')

# Update crs to mactch Cal-Adapt 
school_district = school_district.to_crs('epsg:4326')

ERROR 1: PROJ: proj_create_from_database: Open of /opt/anaconda3/share/proj failed


In [3]:
school_district

,OBJECTID,Year,FedID,CDSCode,CDCode,SCode,CountyName,DistrictNa,SchoolName,SchoolType,...,HOMpct,MIGcount,MIGpct,SEDCount,SEDpct,SWDcount,SWDpct,FRPMcount,FRPMpct,geometry
0,1,2019-20,060161410947,01100170112607,0110017,0112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,High,...,0.0,0,0.0,313.0,76.000000,53,12.900000,295,71.599998,POINT (-122.26838 37.80468)
1,2,2019-20,060169112844,01100170123968,0110017,0123968,Alameda,Alameda County Office of Education,Community School for Creative Education,Elementary,...,0.0,0,0.0,233.0,87.300003,31,11.600000,228,85.400002,POINT (-122.23863 37.78464)
2,3,2019-20,060201312901,01100170124172,0110017,0124172,Alameda,Alameda County Office of Education,Yu Ming Charter,Elementary,...,0.0,0,0.0,79.0,16.200001,28,5.700000,77,15.800000,POINT (-122.28362 37.84737)
3,4,2019-20,060181613008,01100170125567,0110017,0125567,Alameda,Alameda County Office of Education,Urban Montessori Charter,Elementary,...,0.8,0,0.0,136.0,34.599998,41,10.400000,132,33.599998,POINT (-122.18987 37.77846)
4,5,2019-20,060222313772,01100170129403,0110017,0129403,Alameda,Alameda County Office of Education,Epic Charter,Middle,...,0.0,0,0.0,218.0,88.599998,45,18.299999,199,80.900002,POINT (-122.22962 37.77711)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10035,10036,2019-20,064233006928,58727516056840,5872751,6056840,Yuba,Wheatland,Wheatland Elementary,Elementary,...,5.0,0,0.0,189.0,58.700001,32,9.900000,187,58.099998,POINT (-121.42940 39.01167)
10036,10037,2019-20,064233011548,58727516118806,5872751,6118806,Yuba,Wheatland,Wheatland Charter Academy,Elementary,...,0.0,0,0.0,40.0,43.000000,10,10.800000,39,41.900002,POINT (-121.33443 39.10267)
10037,10038,2019-20,064235012741,58727690123570,5872769,0123570,Yuba,Wheatland Union High,Wheatland Community Day High,Community Day,...,0.0,0,0.0,9.0,69.199997,2,15.400000,9,69.199997,POINT (-121.43035 39.00661)
10038,10039,2019-20,064235014385,58727690133751,5872769,0133751,Yuba,Wheatland Union High,Edward P. Duplex,Continuation,...,0.0,0,0.0,31.0,62.000000,10,20.000000,30,60.000000,POINT (-121.43035 39.00661)


In [9]:
# Extract lat and long from shapefile

# Blank data frame
coord_school = pd.DataFrame()

lat_school = school_district['geometry'].x
long_school = school_district['geometry'].y
name = school_district['DistrictNa']

In [10]:
points = []

# lat first then long
# Create a list of POINT's we want the next for loop to iterate over and identify census tracts
for lat, long in zip(lat_school, long_school): 
        point_str = "POINT(" + str(lat) + " " + str(long) + ")"
        points.append(point_str)
        

In [11]:
api = "https://api.cal-adapt.org/api"
headers = {'ContentType': 'json'}

# Name of boundary layer in API (counties track)
resource = "counties"

# Request url
url = '%s/%s/' % (api,resource)

# Request params to find intersecting polygon
all_params = []
all_features = []
full_params = []

for po in points:
    params = {'intersects': po, 'srs':4326, 'simplify':0.0001,'precision':4}
    all_params.append(params)

# Get geometry
for params in all_params:
    response = requests.get(url, params = params, headers = headers)
    if response.ok:
        data = response.json()
        features = data['features'][0]
        all_features.append(features)

unique_ids = set()
        
for feature in all_features:
    if feature:
        ref = '/api/%s/%s/' % (resource, feature['id'])
        
        if feature['id'] not in unique_ids:
            unique_ids.add(feature['id'])
            params = {'ref': ref, 'stat': 'mean'}
            full_params.append(params)
            print('ref:', ref)
            location = 'Counties name %s' % feature['properties']['name']
            print('location:', location) 

ref: /api/counties/1/
location: Counties name Alameda
ref: /api/counties/2/
location: Counties name Alpine
ref: /api/counties/3/
location: Counties name Amador
ref: /api/counties/4/
location: Counties name Butte
ref: /api/counties/5/
location: Counties name Calaveras
ref: /api/counties/6/
location: Counties name Colusa
ref: /api/counties/7/
location: Counties name Contra Costa
ref: /api/counties/39/
location: Counties name San Joaquin
ref: /api/counties/8/
location: Counties name Del Norte
ref: /api/counties/9/
location: Counties name El Dorado
ref: /api/counties/74/
location: Counties name Lyon
ref: /api/counties/10/
location: Counties name Fresno
ref: /api/counties/56/
location: Counties name Ventura
ref: /api/counties/11/
location: Counties name Glenn
ref: /api/counties/12/
location: Counties name Humboldt
ref: /api/counties/13/
location: Counties name Imperial
ref: /api/counties/19/
location: Counties name Los Angeles
ref: /api/counties/14/
location: Counties name Inyo
ref: /api/co

In [12]:
# Base url for extreme heat and %s is the placeholder for the slug
baseurl = 'https://api.cal-adapt.org/api/series/%s/exheat/'

for params in full_params:
    params.update(freq = 'M')

# urlargs: variale numbers of arguments to create combinations for series slugs
# provide additional parameterrs for the API request
# * for lists
# ** for tupples

def fetch_exheat(*urlargs, **params):
    # all possible combinations of the URL
    items = itertools.product(*urlargs)
    series = []
    for parts in items:
        slug = '_'.join(parts)
        print('Processing:', slug)
        # Sends GET request to the API using the URL constructed
        response = requests.get(baseurl % slug, params)
        data = response.json()
        # extracts the counts and converts to pandas series
        counts = pd.Series(data['counts']).sort_index()
        # convert to date time remove time zones
        counts.index = pd.to_datetime(counts.index).tz_localize(None)
        # assign name of the series to the series object
        counts.name = slug
        # append series to list
        series.append(counts)
        # return panda dataframe
    return pd.concat(series, axis=1)

In [13]:
# Base url for extreme heat and %s is the placeholder for the slug
baseurl = 'https://api.cal-adapt.org/api/series/%s/exheat/'

for params in full_params:
    params.update(freq = 'M')

def fetch_exheat(*urlargs, **params):
    items = itertools.product(*urlargs)
    series = []

    for parts in items:
        slug = '_'.join(parts)
        print('Processing:', slug)

        response = requests.get(baseurl % slug, params)

        try:
            response.raise_for_status()
            data = response.json()
        except requests.exceptions.HTTPError as errh:
            print ("HTTP Error:",errh)
        except requests.exceptions.ConnectionError as errc:
            print ("Error Connecting:",errc)
        except requests.exceptions.Timeout as errt:
            print ("Timeout Error:",errt)
        except requests.exceptions.RequestException as err:
            print ("Something went wrong:",err)
        
        try:
            # Your existing code to process the response
            counts = pd.Series(data['counts']).sort_index()
            counts.index = pd.to_datetime(counts.index).tz_localize(None)
            counts.name = slug
            series.append(counts)
        except KeyError as e:
            print(f"Error processing response for {slug}: {e}")
    
    return pd.concat(series, axis=1)

In [46]:
final_df = []

for params in full_params:

    climvar = ['tasmax']
    period = ['day']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['rcp45','rcp85']
    hmodelcounts = fetch_exheat(climvar, period, models, ['historical'], **params)
    projcounts = fetch_exheat(climvar, period, models, scenarios, **params)
    
    for feature in all_features:
        projcounts['county_name'] =  feature['properties']['name']
        hmodelcounts['county_name'] = feature['properties']['name']
    
    projcounts = projcounts.append(hmodelcounts).sort_index()

final_df = pd.concat(projcounts)

Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/4197625489.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/4197625489.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


...

Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/4197625489.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/4197625489.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [79]:
# Create an empty list to store DataFrames for each county
result_dfs = []

for params in full_params:
    climvar = ['tasmax']
    period = ['day']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['rcp45', 'rcp85']

    hmodelcounts = fetch_exheat(climvar, period, models, ['historical'], **params)
    projcounts = fetch_exheat(climvar, period, models, scenarios, **params)
    
    projcounts['county'] = params['ref'][13:16]

    # Append the DataFrames for each county to the list
    result_dfs.append(projcounts.append(hmodelcounts).sort_index())
    
# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(result_dfs)

# Display the final DataFrame
print(final_df.head())


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/930655985.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dfs.append(projcounts.append(hmodelcounts).sort_index())


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/930655985.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dfs.append(projcounts.append(hmodelcounts).sort_index())


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85
Processing: tasmax_day_CanESM2_historical


...

/tmp/ipykernel_1244416/930655985.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dfs.append(projcounts.append(hmodelcounts).sort_index())


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_1244416/930655985.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dfs.append(projcounts.append(hmodelcounts).sort_index())


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85
            tasmax_day_CanESM2_rcp45  tasmax_day_CanESM2_rcp85  \
1950-06-30                       NaN                       NaN   
1950-07-31                       NaN                       NaN   
1950-08-31                       NaN                       NaN   
1950-09-30                       NaN                       NaN   
1950-10-31                       NaN                       NaN   

            tasmax_day_CNRM-CM5_rcp45  tasmax_day_CNRM-CM5_rcp85  \
1950-06-30                        NaN                        N

/tmp/ipykernel_1244416/930655985.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dfs.append(projcounts.append(hmodelcounts).sort_index())


In [73]:
params['ref'][14:16]

'58'

In [80]:
final_df.county.unique()

array([nan, '/1/', '/2/', '/3/', '/4/', '/5/', '/6/', '/7/', '/39', '/8/',
       '/9/', '/74', '/10', '/56', '/11', '/12', '/13', '/19', '/14',
       '/15', '/16', '/17', '/18', '/36', '/30', '/37', '/33', '/20',
       '/22', '/21', '/49', '/23', '/24', '/25', '/47', '/26', '/27',
       '/28', '/29', '/31', '/34', '/32', '/57', '/48', '/35', '/38',
       '/41', '/40', '/42', '/43', '/44', '/45', '/46', '/50', '/51',
       '/52', '/53', '/54', '/55', '/58'], dtype=object)

In [81]:
len(final_df.county.unique())

60

In [15]:
projcounts.to_csv('counties_data_heat.csv', index=False)